In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
plt.style.use('seaborn-white')

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
#torchvision 데이터셋의 출력(output)은 [0, 1] 범위를 갖는 PILImage 이미지이므로 
#이를 [-1, 1]의 범위로 정규화된 Tensor로 변환합니다.

In [3]:
batch_size = 4

trainset = torchvision.datasets.CIFAR10(
    root='./data', 
    train=True,
    download=True, 
    transform=transform
) 
train_loader = torch.utils.data.DataLoader(
    trainset, 
    batch_size=batch_size,
    shuffle=True
)

testset = torchvision.datasets.CIFAR10(
    root='./data', 
    train=False,
    download=True, 
    transform=transform
)
test_loader = torch.utils.data.DataLoader(
    testset, 
    batch_size=batch_size,
    shuffle=False
)

classes = trainset.classes
classes

Files already downloaded and verified
Files already downloaded and verified


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [4]:
class cnn(nn.Module):
    def __init__(self):
        super().__init__()
        #순서대로 in_channels=3, out_channels=6, kernel_size=5, padding=1입니다.
        self.conv1 = nn.Conv2d(3, 6, 5, 1) #rgb의 3색이므로 in_channels=3, padding=1을 주어 정보 축소를 최소화시킵니다.
        self.pool = nn.MaxPool2d(2, 2) #pooling의 방법은 max pooling을 이용합니다 stride값은 2로 줍니다.
        self.conv2 = nn.Conv2d(6, 16, 5, 1)
        self.fc1 = nn.Linear(16 * 5 * 5, 120) 
        self.fc2 = nn.Linear(120, 70)
        self.fc3 = nn.Linear(70, 10)

    def forward(self, x):
        # convoultion # input (32 * 32 * 3)
        x = self.pool(F.relu(self.conv1(x))) # ReLu함수를 이용합니다.
        x = self.pool(F.relu(self.conv2(x))) # input (5 * 5 * 16)
        # fully connected
        x = torch.flatten(x, 1) # 배치를 제외한 모든 차원을 평탄화합니다.
        x = F.relu(self.fc1(x)) # ReLu함수를 이용합니다.
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = cnn()

In [5]:
#loss함수는 CrossEntropy를 이용합니다 pytorch에선 softmax와 
#cross-entropy를 합쳐놓은 것 을 제공하기 때문에 
#맨 마지막 layer가 softmax일 필요가 없습니다. 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) 
#momentum을 주어 localminimum에 빠지기 않게함 

In [6]:
for epoch in range(8):   # 훈련데이터의 훈련을 8번 시킵니다.

    running_loss = 0.0 #오차를 초기화합니다.
    for i, data in enumerate(train_loader, 0):
        # loader 데이터를 가져와서 하나씩 분할합니다.
        inputs, labels = data #왼쪽은 그림, 오른쪽은 분류입니다 각 4개입니다.

        # parameter의 gradient를 0으로 만듭니다.
        optimizer.zero_grad()

        # 순전파와 역전파를 최적화를 합니다. (오차역전파)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() # loss값을 출력합니다.
        if i % 4000 == 3999:    # 4000개의 벤치마다 값을 표기합니다.
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 4000))
            running_loss = 0.0

print('Finished Training')

[1,  4000] loss: 2.058
[1,  8000] loss: 1.628
[1, 12000] loss: 1.498
[2,  4000] loss: 1.384
[2,  8000] loss: 1.343
[2, 12000] loss: 1.310
[3,  4000] loss: 1.230
[3,  8000] loss: 1.200
[3, 12000] loss: 1.194
[4,  4000] loss: 1.099
[4,  8000] loss: 1.111
[4, 12000] loss: 1.112
[5,  4000] loss: 1.023
[5,  8000] loss: 1.035
[5, 12000] loss: 1.047
[6,  4000] loss: 0.967
[6,  8000] loss: 0.969
[6, 12000] loss: 0.994
[7,  4000] loss: 0.898
[7,  8000] loss: 0.937
[7, 12000] loss: 0.942
[8,  4000] loss: 0.865
[8,  8000] loss: 0.903
[8, 12000] loss: 0.905
Finished Training


In [7]:
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH) #학습한 모델을 저장합니다.

In [8]:
model = cnn()
model.load_state_dict(torch.load(PATH)) #저장했던 모델들을 불러옵니다.

<All keys matched successfully>

In [9]:
correct = 0
total = 0
# 출력에 대한 변화도 계산의 필요가 없음(학습 중X)
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # 신경망에 이미지를 값을 입력하여 출력을 계산합니다.
        outputs = model(images)
        # 가장 높은 값를 갖는 분류(class)를 정답으로 선택합니다.
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy : %d %%' % (
    100 * correct / total))

Accuracy : 63 %


In [10]:
# 분류에 대한 예측값 계산을 준비합니다.
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# 변화도는 필요하지 않습니다.
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # 분류별로 맞춘 예측 수를 정렬합니다.
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# 분류별 accuracy를 출력합니다
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

Accuracy airplane is: 78.3 %
Accuracy automobile is: 74.3 %
Accuracy bird  is: 44.3 %
Accuracy cat   is: 45.0 %
Accuracy deer  is: 65.3 %
Accuracy dog   is: 44.9 %
Accuracy frog  is: 75.3 %
Accuracy horse is: 65.8 %
Accuracy ship  is: 73.7 %
Accuracy truck is: 71.8 %


## 보고서

In [ ]:
#CNN을 이용하여 CIFar-10 이미지를 분류하는 작업, 방법을 running시키는것이 목적이다.
#우선 작업에 쓰일 pytorch와 각종 라이브러리를 불러오고 torchvision으로 부터
#데이터들을 불러온다. torchvision 데이터의 출력은 [0, 1] 범위를 갖는 PILImage 이미지 
#이므로 우선 이를 [-1, 1]의 범위로 정규화된 Tensor로 변환한다.
#banch size는 4로 설정해주고(데이터 처리량이 많으므로) dataset으로 부터 load되는
#데이터들을 설정해준다. (셔플과 정규화 텐서로 변환) 해당 이미지는 컬러이므로 
#in channel을 3으로 설정하고 kernel 행렬은 5x5, pading은 1, kernel의 stride값은 2로
#설정해준다. 이때 pooling은 max pooling 방법을 이용해주며 convoultion이 끝날 때마다
#출력되는 out channel은 각각 6, 16으로 해준다 convoultion과 fully connected은
#ReLu함수가 적합하다고 판단하여 해당 함수를 이용해주었고 loss함수는 CrossEntropy를 
#이용해주었다 이때 localminimum에 빠지기 않게하기 위해 mometum값을 주었다.
#훈련은 8번 이상부터 loss값이 수렴하여 그 이상은 의미가 없다 판단하여 8회 학습을
#시켜주었고 오차역전파를 해주어 정확도를 올려줍니다. 최종적으로 출력값이 도출될 때
#출력에서 가장 높은 값을 갖는 분류를 정답으로 선택해주며 전체적인 정답률을 도츌하고
#각 분류별로 맞춘 예측 수를 이용하여 분류별로 맞춘 정답률을 기재해줍니다.
#해당 결과로 미루어보아 cnn을 이용한 이미지 분류는 모든 이미지가 동등한 정답률을
#갖지않음을 알 수 있습니다. 또한 각 분류 별로 정답률이 크게 차이가 나는 것을 볼 때
#정답률이 낮은 그룹과 높은 그룹을 분석하여 해당 사유에 대해 고찰하면 두 그룹이 갖는
#두드러진 공통점과 차이점, 혹은 각각의 한계에 대해 알 수 있으므로 cnn의 성능 개선에
#큰 영향을 줄 것으로 판단됩니다.